In [0]:
%sql
CREATE OR REPLACE VIEW formula_1.int_transform_laps_data AS
WITH duration_sector_1_group AS (
  SELECT
    full_name,
    session_name,
    AVG(duration_sector_1) AS duration_sector_1_mean
  FROM
    formula_1.stg_laps_by_drivers_sessions_meetings
  GROUP BY
    full_name,
    session_name
),
duration_sector_2_group AS (
  SELECT
    full_name,
    session_name,
    AVG(duration_sector_2) AS duration_sector_2_mean
  FROM
    formula_1.stg_laps_by_drivers_sessions_meetings
  GROUP BY
    full_name,
    session_name
),
duration_sector_3_group AS (
  SELECT
    full_name,
    session_name,
    AVG(duration_sector_3) AS duration_sector_3_mean
  FROM
    formula_1.stg_laps_by_drivers_sessions_meetings
  GROUP BY
    full_name,
    session_name
),
i1_speed_group AS (
  SELECT
    full_name,
    session_name,
    MEDIAN(i1_speed) AS i1_speed_median
  FROM
    formula_1.stg_laps_by_drivers_sessions_meetings
  GROUP BY
    full_name,
    session_name
),
i2_speed_group AS (
  SELECT
    full_name,
    session_name,
    MEDIAN(i2_speed) AS i2_speed_median
  FROM
    formula_1.stg_laps_by_drivers_sessions_meetings
  GROUP BY
    full_name,
    session_name
),
st_speed_group AS (
  SELECT
    full_name,
    session_name,
    MEDIAN(st_speed) AS st_speed_median
  FROM
    formula_1.stg_laps_by_drivers_sessions_meetings
  GROUP BY
    full_name,
    session_name
)
SELECT
  stg_laps.meeting_name,
  stg_laps.meeting_official_name,
  stg_laps.meeting_code,
  stg_laps.session_name,
  stg_laps.session_type,
  stg_laps.circuit_short_name,
  stg_laps.meeting_country_code,
  stg_laps.country_name,
  stg_laps.location,
  stg_laps.meeting_date_start,
  stg_laps.session_date_start,
  stg_laps.session_date_end,
  stg_laps.gmt_offset,
  stg_laps.first_name,
  stg_laps.last_name,
  stg_laps.full_name,
  stg_laps.broadcast_name,
  stg_laps.driver_country_code,
  stg_laps.team_name,
  stg_laps.lap_number,
  stg_laps.lap_date_start,
  CASE
    WHEN stg_laps.duration_sector_1 IS NULL THEN d_s_1_g.duration_sector_1_mean
    ELSE stg_laps.duration_sector_1
  END AS duration_sector_1,
  CASE
    WHEN stg_laps.duration_sector_2 IS NULL THEN d_s_2_g.duration_sector_2_mean
    ELSE stg_laps.duration_sector_2
  END AS duration_sector_2,
  CASE
    WHEN stg_laps.duration_sector_3 IS NULL THEN d_s_3_g.duration_sector_3_mean
    ELSE stg_laps.duration_sector_3
  END AS duration_sector_3,
  CASE
    WHEN stg_laps.i1_speed IS NULL THEN i1_s_g.i1_speed_median
    ELSE stg_laps.i1_speed
  END AS i1_speed,
  CASE
    WHEN stg_laps.i2_speed IS NULL THEN i2_s_g.i2_speed_median
    ELSE stg_laps.i2_speed
  END AS i2_speed,
  stg_laps.is_pit_out_lap,
  COALESCE(stg_laps.lap_duration, 
         COALESCE(stg_laps.duration_sector_1,0) 
       + COALESCE(stg_laps.duration_sector_2,0) 
       + COALESCE(stg_laps.duration_sector_3,0)) AS lap_duration,
  COALESCE(stg_laps.segments_sector_1, ARRAY(0)) AS segments_sector_1,
  COALESCE(stg_laps.segments_sector_2, ARRAY(0)) AS segments_sector_2,
  COALESCE(stg_laps.segments_sector_3, ARRAY(0)) AS segments_sector_3,
  COALESCE(stg_laps.st_speed, st_sp_g.st_speed_median) AS st_speed
FROM
  formula_1.stg_laps_by_drivers_sessions_meetings stg_laps
    LEFT JOIN duration_sector_1_group d_s_1_g
      ON stg_laps.full_name = d_s_1_g.full_name
      AND stg_laps.session_name = d_s_1_g.session_name
    LEFT JOIN duration_sector_2_group d_s_2_g
      ON stg_laps.full_name = d_s_2_g.full_name
      AND stg_laps.session_name = d_s_2_g.session_name
    LEFT JOIN duration_sector_3_group d_s_3_g
      ON stg_laps.full_name = d_s_3_g.full_name
      AND stg_laps.session_name = d_s_3_g.session_name
    LEFT JOIN i1_speed_group i1_s_g
      ON stg_laps.full_name = i1_s_g.full_name
      AND stg_laps.session_name = i1_s_g.session_name
    LEFT JOIN i2_speed_group i2_s_g
      ON stg_laps.full_name = i2_s_g.full_name
      AND stg_laps.session_name = i2_s_g.session_name
    LEFT JOIN st_speed_group st_sp_g
      ON stg_laps.full_name = st_sp_g.full_name
      AND stg_laps.session_name = st_sp_g.session_name